# Mink usage examples

This notebook demonstrates some examples of using mink and how it interacts with sklearn.

## Imports

In [1]:
import pickle

In [2]:
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [3]:
from mink import NeuralNetClassifier
from mink import NeuralNetRegressor
from mink.layers import DenseLayer
from mink.layers import InputLayer
from mink.updates import SGD

## MNIST classification task

### Data

In [4]:
X, y = make_classification(n_samples=5000)

### Define neural net

Note: No need to specify the shape of the training data, number of classes, or to set softmax nonlinearity. The `NeuralNetClassifier` class takes care of all of that, as is expected from an sklearn estimator.

In [5]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [6]:
net = NeuralNetClassifier(l2, verbose=1)

If we want to change certain parameters after initialization, just use the `set_params` method and the double-underscore notation known from sklearn.

In [7]:
net.set_params(update__learning_rate=0.5)

NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7fa0526824a8>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_training_started=[<mink.handlers.PrintLayerInfo object at 0x7fa05262f780>],
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

### Train

In [8]:
net.fit(X, y, num_epochs=0)

# Neural Network with 4602 learnable parameters

## Layer information

  #  name      size
---  ------  ------
  0  input       20
  1  dense      200
  2  dense        2



NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7fa0526824a8>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_training_started=[<mink.handlers.PrintLayerInfo object at 0x7fa05262f780>],
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [9]:
y_proba = net.predict_proba(X)

In [10]:
(np.argmax(y_proba, 1) == y).mean()

0.34300000000000003

In [11]:
pdb on

Automatic pdb calling has been turned ON


In [12]:
net.fit(X, y, num_epochs=20)

# Neural Network with 4602 learnable parameters

## Layer information

  #  name      size
---  ------  ------
  0  input       20
  1  dense      200
  2  dense        2

epochs:    1 | loss: 0.43984
epochs:    2 | loss: 0.41169
epochs:    3 | loss: 0.38898
epochs:    4 | loss: 0.36973
epochs:    5 | loss: 0.35518
epochs:    6 | loss: 0.34383
epochs:    7 | loss: 0.33445
epochs:    8 | loss: 0.32645
epochs:    9 | loss: 0.31980
epochs:   10 | loss: 0.31451
epochs:   11 | loss: 0.30931
epochs:   12 | loss: 0.30446
epochs:   13 | loss: 0.29904
epochs:   14 | loss: 0.29477
epochs:   15 | loss: 0.29073
epochs:   16 | loss: 0.28681
epochs:   17 | loss: 0.28330
epochs:   18 | loss: 0.27977
epochs:   19 | loss: 0.27669
epochs:   20 | loss: 0.27409


NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7fa0526824a8>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_training_started=[<mink.handlers.PrintLayerInfo object at 0x7fa05262f780>],
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [13]:
y_proba

array([[ 0.54106098,  0.45893902],
       [ 0.54056048,  0.45943949],
       [ 0.62416959,  0.37583038],
       ..., 
       [ 0.48155829,  0.51844168],
       [ 0.41701135,  0.58298868],
       [ 0.57692099,  0.42307904]], dtype=float32)

In [14]:
np.argmax(y_proba, 1)[:10]

array([0, 0, 0, 1, 0, 1, 0, 0, 1, 0])

In [15]:
y_proba = net.predict_proba(X)

In [16]:
(np.argmax(y_proba, 1) == y.flatten()).mean()

0.86160000000000003

### Grid search

The neural net estimators can be used in conjunction with other sklearn features, such as `GridSearchCV`.

In [17]:
l0 = InputLayer()
l1 = DenseLayer(l0, name='hidden')
l2 = DenseLayer(l1)

In [18]:
net = NeuralNetClassifier(l2, update=SGD())

Parameters are set using the known double-underscore notation, e.g.

`'update__learning_rate': [0.1, 0.5]`.

Note: Instead of having to write

`'layer__incoming__num_units': [50, 100]`

we can just write

`'hidden__num_units': [50, 100]`

because we have given the hidden layer a name, "hidden". This may safe a lot of writing and confusion.

In [19]:
params = {
    'update__learning_rate': [0.1, 0.5],
    'max_epochs': [5, 10],
    'hidden__num_units': [50, 100],
}

In [20]:
cv = GridSearchCV(net, params, scoring='accuracy', refit=False, verbose=3)

In [21]:
cv.fit(X, y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] max_epochs=5, update__learning_rate=0.1, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.1, hidden__num_units=50, score=0.806239 -   0.2s
[CV] max_epochs=5, update__learning_rate=0.1, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.1, hidden__num_units=50, score=0.791242 -   0.3s
[CV] max_epochs=5, update__learning_rate=0.1, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.1, hidden__num_units=50, score=0.821729 -   0.3s
[CV] max_epochs=5, update__learning_rate=0.5, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.5, hidden__num_units=50, score=0.816437 -   0.3s
[CV] max_epochs=5, update__learning_rate=0.5, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.5, hidden__num_units=50, score=0.820036 -   0.4s
[CV] max_epochs=5, update__learning_rate=0.5, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.5, hi

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   16.6s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=NeuralNetClassifier(batch_iterator=<mink.nolearn.BatchIterator object at 0x7fa0526824a8>,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=Glor...bject at 0x7fa05262f780>],
          session_kwargs=None, update=SGD(learning_rate=0.01), verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'update__learning_rate': [0.1, 0.5], 'max_epochs': [5, 10], 'hidden__num_units': [50, 100]},
       pre_dispatch='2*n_jobs', refit=False, scoring='accuracy', verbose=3)

In [22]:
cv.best_params_, cv.best_score_

({'hidden__num_units': 50, 'max_epochs': 10, 'update__learning_rate': 0.5},
 0.8236)

## Regression task

As is known from sklearn, we have separate estimators for classification and regression.

### Data

In [23]:
X, y = make_regression(n_samples=5000)

Note that apart from using `NeuralNetRegressor` instead of `NeuralNetClassifier`, everything is the same. No need to adjust output nonlinearity or objective.

### Define neural network

In [24]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [25]:
net = NeuralNetRegressor(l2, verbose=1)

In [26]:
net.set_params(update__learning_rate=0.001)

NeuralNetRegressor(batch_iterator=<mink.nolearn.BatchIterator object at 0x7fa05262f978>,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=MeanSquaredError(),
          on_training_started=[<mink.handlers.PrintLayerInfo object at 0x7fa05262f9b0>],
          session_kwargs=None, update=SGD(learning_rate=0.001), verbose=1)

### Train

In [27]:
net.fit(X, y, num_epochs=20)

# Neural Network with 20401 learnable parameters

## Layer information

  #  name      size
---  ------  ------
  0  input      100
  1  dense      200
  2  dense        1

epochs:    1 | loss: 7652.36719
epochs:    2 | loss: 107.46196
epochs:    3 | loss: 78.80222
epochs:    4 | loss: 63.64157
epochs:    5 | loss: 52.32061
epochs:    6 | loss: 43.18874
epochs:    7 | loss: 35.97786
epochs:    8 | loss: 30.09224
epochs:    9 | loss: 25.29941
epochs:   10 | loss: 21.33082
epochs:   11 | loss: 18.05920
epochs:   12 | loss: 15.31239
epochs:   13 | loss: 13.05539
epochs:   14 | loss: 11.20075
epochs:   15 | loss: 9.65204
epochs:   16 | loss: 8.36928
epochs:   17 | loss: 7.29081
epochs:   18 | loss: 6.39702
epochs:   19 | loss: 5.65079
epochs:   20 | loss: 5.03053


NeuralNetRegressor(batch_iterator=<mink.nolearn.BatchIterator object at 0x7fa05262f978>,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(val=0.0),
      incoming=InputLayer(Xs=None, name=None, ys=None), name=None,
      nonlinearity=None, num_units=200),
      name=None, nonlinearity=Linear(), num_units=1),
          max_epochs=10, objective=MeanSquaredError(),
          on_training_started=[<mink.handlers.PrintLayerInfo object at 0x7fa05262f9b0>],
          session_kwargs=None, update=SGD(learning_rate=0.001), verbose=1)

## Saving and restoring

### Save previous net

In [28]:
score_before = mean_squared_error(y, net.predict(X))
print(score_before)

9.43870165903


In [29]:
with open('mynet.pkl', 'wb') as f:
    pickle.dump(net, f)

### Create a new net with same architecture

In [30]:
with open('mynet.pkl', 'rb') as f:
    new_net = pickle.load(f)

In [31]:
new_net.initialize(X, y)

In [32]:
score_after = mean_squared_error(y, new_net.predict(X))
print(score_after)
assert np.isclose(score_before, score_after)

9.43870165903
